In [ ]:
# This script is used to collect experiments' results and generate a summary table
# input: path to the directory containing the results, specific cls_name to include in the summary table 
# output: show a summary table and optionally save it to a csv file

In [ ]:
# import package
from pathlib import Path
import pandas as pd
import json
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

In [ ]:
# input
output_folder = './output'
cls_name = 'overall' # 'overall', 'background', or specific cls_name

## One Dataset Per Row

In [ ]:
output = Path(output_folder)
dataset_items = {
    "Experiment": [],
    "Dataset": [],
    "mIoU": [],
    "mIoU(dcrf)": [],
    "m_TP_T": [],
    "m_TP_T(dcrf)": [],
    "m_TP_P": [],
    "m_TP_P(dcrf)": [],
    "Threshold": [],
}
for experiment in output.iterdir():
    for dataset in experiment.iterdir():
        result_path = dataset / "segmentation_metrics.json"
        dcrf_result_path = dataset / "segmentation_metrics_crf.json"
        if not result_path.exists():
            continue
        with open(result_path) as f:
            result = json.load(f)
            if cls_name not in result:
                continue
            dataset_items["Experiment"].append(experiment.name)
            dataset_items["Dataset"].append(dataset.name)
            dataset_items["mIoU"].append(round(result[cls_name]["m_IoU"][0],2))
            dataset_items["m_TP_T"].append(round(result[cls_name]["m_TP_T"][0],2))
            dataset_items["m_TP_P"].append(round(result[cls_name]["m_TP_P"][0],2))
            dataset_items["Threshold"].append(result["best threshold"])
        if dcrf_result_path.exists():
            with open(dcrf_result_path) as f:
                result = json.load(f)
                dataset_items["mIoU(dcrf)"].append(round(result[cls_name]["m_IoU"][0],2))
                dataset_items["m_TP_T(dcrf)"].append(round(result[cls_name]["m_TP_T"][0],2))
                dataset_items["m_TP_P(dcrf)"].append(round(result[cls_name]["m_TP_P"][0],2))
        else:
            dataset_items["mIoU(dcrf)"].append(None)
            dataset_items["m_TP_T(dcrf)"].append(None)
            dataset_items["m_TP_P(dcrf)"].append(None)
dataset_table = pd.DataFrame(dataset_items)
dataset_table

## One Experiment Per Row

In [ ]:
items_map = {}  # first dim is experiment, second dim is keyword
dataset_names = ["voc_sim", "coco_cap", "voc", "context", "coco"]
for idx in range(len(dataset_table)):
    # 1. get experiment name
    experiment = dataset_items["Experiment"][idx]
    if experiment not in items_map:
        items_map[experiment] = {}
    # 2. get dataset name and keyword
    dataset = dataset_items["Dataset"][idx]
    for name in dataset_names:
        if name in dataset:
            keyword = dataset.replace(name, "")
            dataset = name
            break
    if keyword not in items_map[experiment]:
        items_map[experiment][keyword] = {
            "voc": None,
            "context": None,
            "coco": None,
            "voc_sim": None,
            "coco_cap": None,
        }
    # 3. record the result
    items_map[experiment][keyword][dataset] = dataset_items["mIoU"][idx]
    # 4. record the result with dcrf
    if dataset_items["mIoU(dcrf)"][idx] is not None:
        keyword = keyword + ", dcrf" if keyword else "dcrf"
        if keyword not in items_map[experiment]:
            items_map[experiment][keyword] = {
                "voc": None,
                "context": None,
                "coco": None,
                "voc_sim": None,
                "coco_cap": None,
            }
        items_map[experiment][keyword][dataset] = dataset_items["mIoU(dcrf)"][idx]
experiment_items = {
    "Experiment": [],
    "voc": [],
    "context": [],
    "coco": [],
    "voc_sim": [],
    "coco_cap": [],
    "keyword": [],
}
for experiment in items_map:
    for keyword in items_map[experiment]:
        experiment_items["Experiment"].append(experiment)
        experiment_items["keyword"].append(keyword)
        for dataset in dataset_names:
            experiment_items[dataset].append(items_map[experiment][keyword][dataset])
experiment_table = pd.DataFrame(experiment_items)
experiment_table